In [ ]:
import pandas as pd
import os
import random

# Loading beam data
beam_data = pd.read_csv('structural-sections_3.csv')

# Specifying the output directory
output_directory = 'C:\\Users\\Misiek\\Desktop\\Kody Do Milestone 2\\'
os.makedirs(output_directory, exist_ok=True)
output_file_path = os.path.join(output_directory, 'selected_beams_3.csv')

# Function to save selected beam data
def save_selected_beam_data(selected_beam, user_input_data, output_file_path=output_file_path):
    input_data = pd.DataFrame([user_input_data], columns=['Applied Force Vertical', 'Applied Force Horizontal',
                                                           'Length', 'Gamma_M', 'Steel Class', 'Depth', 'Width', 'Deflection Limit'])

    exclude_columns = ['Moment Resistance y-y', 'Moment Resistance z-z', 'Difference y-y',
                       'Difference z-z', 'Deflection y-y', 'Deflection z-z']
    selected_beam_filtered = selected_beam.drop(exclude_columns, axis=1)
    selected_beam_with_input = pd.concat([selected_beam_filtered, input_data], axis=1)
    
    selected_beam_with_input.to_csv(output_file_path, mode='a', index=False, header=not os.path.exists(output_file_path))

# Loop for selecting beams and saving data
for _ in range(100000):
    # Function for user data input
    applied_force_vertical = random.randint(1, 100)
    applied_force_horizontal = random.randint(1, 100)
    length = random.randint(1, 25)
    gamma_M = 1.15
    steel_class = 355
    deflection_limit = length * 1000 / 360
    depth = 1000
    width = 1000

    # Function for calculating moments
    def moment_calc(applied_force_vertical, applied_force_horizontal, length):
        moment_y_y = (1/4) * applied_force_vertical * length
        moment_z_z = (1/4) * applied_force_horizontal * length
        return moment_y_y, moment_z_z

    # Function for calculating shear force
    def shear_force_calc(applied_force_vertical):
        shear_force_z_z = applied_force_vertical / 2
        return shear_force_z_z

    # Function for calculating moment resistance
    def moment_resistance_calc(steel_class, gamma_M):
        beam_data['Moment Resistance y-y'] = (beam_data['Plastic Modulus y-y'] * steel_class) / gamma_M / 1000
        beam_data['Moment Resistance z-z'] = (beam_data['Plastic Modulus z-z'] * steel_class) / gamma_M / 1000

    # Function for calculating shear area
    def shear_area_calc():
        beam_data['Shear_Area'] = beam_data['Thickness Web'] * beam_data['Depth of section'] - 2 * beam_data['Thickness Flange']

    # Function for calculating shear resistance
    def shear_resistance_calc(steel_class, gamma_M):
        beam_data['Shear Resistance z-z'] = (beam_data['Shear_Area'] * steel_class) / (3 ** (1/2) * gamma_M)

    # Function for calculating deflection
    def calculate_deflection(applied_force, length, elastic_modulus, moment_of_inertia):
        deflection = (applied_force * length**3) * 1000 / (48 * elastic_modulus * moment_of_inertia)
        return deflection

    # Function for calculating deflections
    def calculate_deflections(applied_force_vertical, applied_force_horizontal, length):
        beam_data['Deflection y-y'] = calculate_deflection(applied_force_vertical, length * 1000, 210000,
                                                           beam_data['Second Moment of Area y-y'] * 10000)
        beam_data['Deflection z-z'] = calculate_deflection(applied_force_horizontal, length * 1000, 210000,
                                                           beam_data['Second Moment of Area z-z'] * 10000)

    # Function for selecting a beam
    def selected_beam(moment_y_y, moment_z_z, deflection_limit, shear_force_z_z, depth, width):
        beam_data['Difference y-y'] = abs(beam_data['Moment Resistance y-y'] - moment_y_y)
        beam_data['Difference z-z'] = abs(beam_data['Moment Resistance z-z'] - moment_z_z)

        beam_data_filtered = beam_data[(beam_data['Deflection y-y'] < deflection_limit) &
                                       (beam_data['Deflection z-z'] < deflection_limit) &
                                       (beam_data['Shear Resistance z-z'] > shear_force_z_z)]
        beam_data_filtered = beam_data_filtered[(beam_data_filtered['Depth of section'] < depth) &
                                                (beam_data_filtered['Width of section'] < width)]

        selected_beam = beam_data_filtered.nsmallest(1, ['Difference y-y', 'Difference z-z'])

        return selected_beam[['Beam_Name', 'Moment Resistance y-y', 'Moment Resistance z-z', 'Difference y-y',
                              'Difference z-z', 'Deflection y-y', 'Deflection z-z']]

    # Function for displaying calculated data
    def display_calculated_data(selected_beam, moment_y_y, moment_z_z, shear_force_z_z, user_input_data):
        print("\nTop beam with estimated moment resistance greater than applied moment (y-y and z-z):\n" +
              str(selected_beam[['Beam_Name', 'Moment Resistance y-y', 'Deflection y-y',
                                 'Moment Resistance z-z', 'Deflection z-z']]) +
              f"\n\nCalculated Moment (y-y): {moment_y_y}\nCalculated Moment (z-z): {moment_z_z}"
              f"\nCalculated Shear Force (z-z): {shear_force_z_z}")
        if not selected_beam.empty:
            # Save to file
            save_selected_beam_data(selected_beam, user_input_data)

    # User data selection
    user_input_data = (applied_force_vertical, applied_force_horizontal, length, gamma_M, steel_class, depth, width, deflection_limit)
    moments = moment_calc(applied_force_vertical, applied_force_horizontal, length)
    shear_force = shear_force_calc(applied_force_vertical)
    moment_resistance_calc(steel_class, gamma_M)
    shear_area_calc()
    shear_resistance_calc(steel_class, gamma_M)
    calculate_deflections(applied_force_vertical, applied_force_horizontal, length)
    selected_beam_data = selected_beam(*moments, deflection_limit, shear_force, depth, width)

    display_calculated_data(selected_beam_data, *moments, shear_force, user_input_data)

# Read data from file
output_data = pd.read_csv(output_file_path)
print(f"Output data saved to {output_file_path}")
print("User input data:", user_input_data)



Top beam with estimated moment resistance greater than applied moment (y-y and z-z):
Empty DataFrame
Columns: [Beam_Name, Moment Resistance y-y, Deflection y-y, Moment Resistance z-z, Deflection z-z]
Index: []

Calculated Moment (y-y): 518.75
Calculated Moment (z-z): 300.0
Calculated Shear Force (z-z): 41.5

Top beam with estimated moment resistance greater than applied moment (y-y and z-z):
           Beam_Name  Moment Resistance y-y  Deflection y-y  \
210  350  x  350 x 6             324.130435       20.792411   

     Moment Resistance z-z  Deflection z-z  
210             324.130435       24.408482  

Calculated Moment (y-y): 103.5
Calculated Moment (z-z): 121.5
Calculated Shear Force (z-z): 23.0

Top beam with estimated moment resistance greater than applied moment (y-y and z-z):
           Beam_Name  Moment Resistance y-y  Deflection y-y  \
371  500  x  200 x 8             527.869565       32.693452   

     Moment Resistance z-z  Deflection z-z  
371             276.591304     


Top beam with estimated moment resistance greater than applied moment (y-y and z-z):
            Beam_Name  Moment Resistance y-y  Deflection y-y  \
219  400  x  400 x 10             682.217391       28.344449   

     Moment Resistance z-z  Deflection z-z  
219             682.217391       29.035777  

Calculated Moment (y-y): 225.5
Calculated Moment (z-z): 231.0
Calculated Shear Force (z-z): 41.0

Top beam with estimated moment resistance greater than applied moment (y-y and z-z):
Empty DataFrame
Columns: [Beam_Name, Moment Resistance y-y, Deflection y-y, Moment Resistance z-z, Deflection z-z]
Index: []

Calculated Moment (y-y): 93.5
Calculated Moment (z-z): 272.0
Calculated Shear Force (z-z): 11.0

Top beam with estimated moment resistance greater than applied moment (y-y and z-z):
           Beam_Name  Moment Resistance y-y  Deflection y-y  \
377  500  x  300 x 8              648.26087       36.776724   

     Moment Resistance z-z  Deflection z-z  
377             456.869565     


Top beam with estimated moment resistance greater than applied moment (y-y and z-z):
Empty DataFrame
Columns: [Beam_Name, Moment Resistance y-y, Deflection y-y, Moment Resistance z-z, Deflection z-z]
Index: []

Calculated Moment (y-y): 336.0
Calculated Moment (z-z): 162.75
Calculated Shear Force (z-z): 32.0

Top beam with estimated moment resistance greater than applied moment (y-y and z-z):
           Beam_Name  Moment Resistance y-y  Deflection y-y  \
292  200  x  120 x 5              63.282609        4.393424   

     Moment Resistance z-z  Deflection z-z  
292              44.452174        5.103529  

Calculated Moment (y-y): 46.5
Calculated Moment (z-z): 24.5
Calculated Shear Force (z-z): 46.5

Top beam with estimated moment resistance greater than applied moment (y-y and z-z):
           Beam_Name  Moment Resistance y-y  Deflection y-y  \
144  70  x  70 x 3.5               6.853043        2.590642   

     Moment Resistance z-z  Deflection z-z  
144               6.853043       


Top beam with estimated moment resistance greater than applied moment (y-y and z-z):
            Beam_Name  Moment Resistance y-y  Deflection y-y  \
180  160  x  160 x  5              54.021739        4.695767   

     Moment Resistance z-z  Deflection z-z  
180              54.021739        2.050265  

Calculated Moment (y-y): 35.5
Calculated Moment (z-z): 15.5
Calculated Shear Force (z-z): 35.5

Top beam with estimated moment resistance greater than applied moment (y-y and z-z):
Empty DataFrame
Columns: [Beam_Name, Moment Resistance y-y, Deflection y-y, Moment Resistance z-z, Deflection z-z]
Index: []

Calculated Moment (y-y): 258.0
Calculated Moment (z-z): 456.0
Calculated Shear Force (z-z): 21.5

Top beam with estimated moment resistance greater than applied moment (y-y and z-z):
Empty DataFrame
Columns: [Beam_Name, Moment Resistance y-y, Deflection y-y, Moment Resistance z-z, Deflection z-z]
Index: []

Calculated Moment (y-y): 4.0
Calculated Moment (z-z): 340.0
Calculated Shear F


Top beam with estimated moment resistance greater than applied moment (y-y and z-z):
           Beam_Name  Moment Resistance y-y  Deflection y-y  \
204  300  x  300 x 6             235.843478       14.199656   

     Moment Resistance z-z  Deflection z-z  
204             235.843478        5.378657  

Calculated Moment (y-y): 99.0
Calculated Moment (z-z): 37.5
Calculated Shear Force (z-z): 33.0

Top beam with estimated moment resistance greater than applied moment (y-y and z-z):
           Beam_Name  Moment Resistance y-y  Deflection y-y  \
292  200  x  120 x 5              63.282609        6.218112   

     Moment Resistance z-z  Deflection z-z  
292              44.452174        7.030371  

Calculated Moment (y-y): 29.25
Calculated Moment (z-z): 15.0
Calculated Shear Force (z-z): 19.5

Top beam with estimated moment resistance greater than applied moment (y-y and z-z):
           Beam_Name  Moment Resistance y-y  Deflection y-y  \
218  400  x  400 x 8             555.652174       25

...


Top beam with estimated moment resistance greater than applied moment (y-y and z-z):
Empty DataFrame
Columns: [Beam_Name, Moment Resistance y-y, Deflection y-y, Moment Resistance z-z, Deflection z-z]
Index: []

Calculated Moment (y-y): 140.0
Calculated Moment (z-z): 343.0
Calculated Shear Force (z-z): 20.0

Top beam with estimated moment resistance greater than applied moment (y-y and z-z):
              Beam_Name  Moment Resistance y-y  Deflection y-y  \
221  400  x  400 x 12.5             827.304348       33.239963   

     Moment Resistance z-z  Deflection z-z  
221             827.304348       19.047619  

Calculated Moment (y-y): 267.0
Calculated Moment (z-z): 153.0
Calculated Shear Force (z-z): 44.5

Top beam with estimated moment resistance greater than applied moment (y-y and z-z):
Empty DataFrame
Columns: [Beam_Name, Moment Resistance y-y, Deflection y-y, Moment Resistance z-z, Deflection z-z]
Index: []

Calculated Moment (y-y): 556.25
Calculated Moment (z-z): 206.25
Calculat


Top beam with estimated moment resistance greater than applied moment (y-y and z-z):
           Beam_Name  Moment Resistance y-y  Deflection y-y  \
204  300  x  300 x 6             235.843478       11.330042   

     Moment Resistance z-z  Deflection z-z  
204             235.843478       11.579054  

Calculated Moment (y-y): 113.75
Calculated Moment (z-z): 116.25
Calculated Shear Force (z-z): 45.5

Top beam with estimated moment resistance greater than applied moment (y-y and z-z):
           Beam_Name  Moment Resistance y-y  Deflection y-y  \
198  250  x  250 x 6             161.756522        5.486968   

     Moment Resistance z-z  Deflection z-z  
198             161.756522        9.182274  

Calculated Moment (y-y): 49.0
Calculated Moment (z-z): 82.0
Calculated Shear Force (z-z): 24.5

Top beam with estimated moment resistance greater than applied moment (y-y and z-z):
           Beam_Name  Moment Resistance y-y  Deflection y-y  \
212  350  x  350 x 8             422.913043      


Top beam with estimated moment resistance greater than applied moment (y-y and z-z):
Empty DataFrame
Columns: [Beam_Name, Moment Resistance y-y, Deflection y-y, Moment Resistance z-z, Deflection z-z]
Index: []

Calculated Moment (y-y): 26.25
Calculated Moment (z-z): 255.0
Calculated Shear Force (z-z): 3.5

Top beam with estimated moment resistance greater than applied moment (y-y and z-z):
Empty DataFrame
Columns: [Beam_Name, Moment Resistance y-y, Deflection y-y, Moment Resistance z-z, Deflection z-z]
Index: []

Calculated Moment (y-y): 150.0
Calculated Moment (z-z): 305.0
Calculated Shear Force (z-z): 15.0

Top beam with estimated moment resistance greater than applied moment (y-y and z-z):
Empty DataFrame
Columns: [Beam_Name, Moment Resistance y-y, Deflection y-y, Moment Resistance z-z, Deflection z-z]
Index: []

Calculated Moment (y-y): 220.0
Calculated Moment (z-z): 400.0
Calculated Shear Force (z-z): 27.5

Top beam with estimated moment resistance greater than applied moment (y-


Top beam with estimated moment resistance greater than applied moment (y-y and z-z):
Empty DataFrame
Columns: [Beam_Name, Moment Resistance y-y, Deflection y-y, Moment Resistance z-z, Deflection z-z]
Index: []

Calculated Moment (y-y): 37.5
Calculated Moment (z-z): 236.25
Calculated Shear Force (z-z): 5.0

Top beam with estimated moment resistance greater than applied moment (y-y and z-z):
           Beam_Name  Moment Resistance y-y  Deflection y-y  \
185  180  x  180 x 6              81.495652        0.656513   

     Moment Resistance z-z  Deflection z-z  
185              81.495652        8.140756  

Calculated Moment (y-y): 3.75
Calculated Moment (z-z): 46.5
Calculated Shear Force (z-z): 2.5

Top beam with estimated moment resistance greater than applied moment (y-y and z-z):
             Beam_Name  Moment Resistance y-y  Deflection y-y  \
211  350  x  350 x 6.3             336.478261         3.48537   

     Moment Resistance z-z  Deflection z-z  
211             336.478261      


Top beam with estimated moment resistance greater than applied moment (y-y and z-z):
Empty DataFrame
Columns: [Beam_Name, Moment Resistance y-y, Deflection y-y, Moment Resistance z-z, Deflection z-z]
Index: []

Calculated Moment (y-y): 384.0
Calculated Moment (z-z): 516.0
Calculated Shear Force (z-z): 32.0

Top beam with estimated moment resistance greater than applied moment (y-y and z-z):
            Beam_Name  Moment Resistance y-y  Deflection y-y  \
378  500  x  300 x 10             802.608696       43.807164   

     Moment Resistance z-z  Deflection z-z  
378             564.913043        9.992194  

Calculated Moment (y-y): 232.0
Calculated Moment (z-z): 24.0
Calculated Shear Force (z-z): 29.0

Top beam with estimated moment resistance greater than applied moment (y-y and z-z):
            Beam_Name  Moment Resistance y-y  Deflection y-y  \
219  400  x  400 x 10             682.217391        22.46816   

     Moment Resistance z-z  Deflection z-z  
219             682.217391   